In [1]:
import platform, sys, os


# Variables

In [2]:
## Seed for the random variables
seed_number=0;

BATCH_SIZE=32

## Model of network
#model_type = 'efficientnet_b3'
model_type = 'mobilenet_v3';
#model_type = 'inception_v3';
#model_type = 'inception_resnet_v2';
#model_type = 'resnet_v2_50';

times=1;


# Endereços

In [3]:
## Dataset 
if platform.system()=='Linux':
    if os.getenv("COLAB_RELEASE_TAG"):
        dataset_base_dir    = 'fer2013/archive/test';
    elif 'microsoft-standard' in platform.uname().release:
        dataset_base_dir    = '/mnt/c/Dados/Fernando/DATASET/fer2013/archive/test';
    else:
        dataset_base_dir    = '/mnt/boveda/DATASETs/FACE-EMOTION/fer2013/archive/test';
else:
    dataset_base_dir    = 'C:\\Dados\\Fernando\\DATASET\\fer2013\\archive\\test';

print('dataset_base_dir:',dataset_base_dir)

## Output
if platform.system()=='Linux':
    if os.getenv("COLAB_RELEASE_TAG"):
        output_base_dir = 'OUTPUTS/cnn_face_emotion/test_holdout';
    elif 'microsoft-standard' in platform.uname().release:
        output_base_dir = '/mnt/c/Dados/Fernando/OUTPUTS/cnn_face_emotion/test_holdout';
    else:
        output_base_dir = '/mnt/boveda/DOCTORADO2/cnn_face_emotion/test_holdout';
else:
    output_base_dir = 'C:\\Dados\\Fernando\\OUTPUTS\\cnn_face_emotion\\test_holdout';

print('output_base_dir:',output_base_dir)


# Best model
if platform.system()=='Linux':
    if os.getenv("COLAB_RELEASE_TAG"):
        path_of_best_model_file = 'cnn_face_emotion/library/models';
    elif 'microsoft-standard' in platform.uname().release:
        path_of_best_model_file = '/mnt/c/Dados/Fernando/CODE/PESQUISA/programs-tese/cnn_face_emotion/library/models';
    else:
        path_of_best_model_file='/home/fernando/Downloads/TESIS-DOUTORADO-2/PESQUISA/programs-tese/cnn_face_emotion/library/models';
else:
    path_of_best_model_file='C:\\Dados\\Fernando\\CODE\\PESQUISA\\programs-tese\\cnn_face_emotion\\library\\models';

print('path_of_best_model_file:',path_of_best_model_file)

dataset_base_dir: /mnt/boveda/DATASETs/FACE-EMOTION/fer2013/archive/test
output_base_dir: /mnt/boveda/DOCTORADO2/cnn_face_emotion/test_holdout
path_of_best_model_file: /home/fernando/Downloads/TESIS-DOUTORADO-2/PESQUISA/programs-tese/cnn_face_emotion/library/models


# Bibliotecas externas

In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf

# Biblioteca local

In [5]:
sys.path.append('library');

# If command line

In [6]:
#print('cmd entry:', sys.argv)

for n in range(len(sys.argv)):
    if sys.argv[n]=='--model':
        model_type=sys.argv[n+1];
    if sys.argv[n]=='--times':
        times=int(sys.argv[n+1]);


best_model_file=os.path.join(path_of_best_model_file,'modelo_'+model_type+'.h5');

print('model_type:',model_type)
print('times:',times)

model_type: mobilenet_v3
times: 1


# Set seed of random variables


In [7]:
np.random.seed(seed_number)
tf.keras.utils.set_random_seed(seed_number);

# Data augmentation configuration

In [8]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

idg_test= ImageDataGenerator(rescale=1./255 )


# Creating output directory

In [9]:
#output_dir = os.path.join(output_base_dir,'holdout');
output_dir = os.path.join(output_base_dir,model_type);

try: 
    os.makedirs(output_dir) 
except: 
    pass


# Create new model

In [10]:
import lib_model as mpp

model, target_size = mpp.create_model('',model_type=model_type);
model.summary()

mpp.save_model_parameters(model, os.path.join(output_dir,'parameters_stats.m'));


Loaded layer with mobilenet_v3
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1024)              1529968   
                                                                 
 dense (Dense)               (None, 32)                32800     
                                                                 
 dense_1 (Dense)             (None, 7)                 231       
                                                                 
Total params: 1,562,999
Trainable params: 1,550,887
Non-trainable params: 12,112
_________________________________________________________________


# Defining directories

In [11]:

test_data_generator  = idg_test.flow_from_directory(dataset_base_dir, 
                                                    target_size=target_size,
                                                    batch_size=BATCH_SIZE,
                                                    shuffle=True
                                                    );

Found 7173 images belonging to 7 classes.


# Compile


In [12]:
# COMPILE NEW MODEL
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['categorical_accuracy'])


# Time

In [13]:
from pytictoc import TicToc
t = TicToc() 

# Evaluate best model

In [14]:
# LOAD BEST MODEL to evaluate the performance of the model
model.load_weights(best_model_file);


L=test_data_generator.samples;
print('L:',L);

t.tic();
for m in range(times):
    results = model.evaluate(test_data_generator,verbose=0)
t0=t.tocvalue();

results = dict(zip(model.metrics_names,results))
print(results,"\n\n");


with open(os.path.join(output_dir,"results_testing.m"), 'w') as f: 
    for key, value in results.items(): 
        f.write('%s=%s;\n' % (key, value));
    f.write('delayms=%s;\n' % (t0*1000.0/(times*L)));

tf.keras.backend.clear_session()


L: 7173
{'loss': 1.3863438367843628, 'categorical_accuracy': 0.6750313639640808} 


